<a href="https://colab.research.google.com/github/Giogia/Scene-Reconstruction/blob/master/tsdf_fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
pip install pycuda OpenEXR open3d-python plotly

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 3.7MB 43.1MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 81kB 11.3MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4533536 sha256=c5158a0136b87321443d8b63efed16dd30ac6a5b2fae9d3051d6b0c2d3786ded
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for OpenEXR: filename=OpenEXR-1.3.2-cp36-cp36m-linux_x86_64.whl size=188420 sha256=de8b3813e3d6606f2956f8d0de8c845bf779ed50857c42b0673eb60fd6e0238c
  Stored in directory: /root/.cache/pip/wheels/41/06/9f/c7dc838815b0e7dfc7d7dc19cc3d677edb47594d8489adc62a
  Created wheel for pytools: filename=pytools-2020.1-py2.py3-none-any.whl size=59602 sha256=970eb453660fef211ee895011e87b78ea7005dea58ec1112a96e125bbaba8845
  Stored in directory: /root/.cache/pip/wheels/6f/da/1b/946775a88291378182ed92c

In [0]:
import numpy as np

%matplotlib inline 
from matplotlib import pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [18]:
import sys
sys.path.append('/content/gdrive/My Drive/Colab')

from csv_utils import read_csv
from exr_utils import exr_to_image, exr_to_depth
from tsdf_fusion import get_view_frustum, TSDFVolume, meshwrite

ModuleNotFoundError: ignored

In [0]:
def show_array(data):
    data /= np.max(data)
    plt.imshow(data, interpolation='bicubic')
    plt.show()

In [0]:
from open3d import read_point_cloud
import plotly.express as px

def show_mesh(name):
    cloud = read_point_cloud(name + ".ply")
    array = np.asarray(cloud.points)
    print(array.shape)
    
    df = px.data.iris()
    fig = px.scatter_3d(df, x='sepal_length', y='sepal_width', z='petal_width',
              color='species')
    fig.show()

In [0]:
IMAGES_NUMBER = 4

In [0]:
def fuse():

    print("Estimating voxel volume bounds...")
    images = IMAGES_NUMBER

    camera_intrinsics = np.reshape(read_csv(file='camera_calibration.csv', field='Camera Intrinsics'), (3, 3))

    bounds = np.zeros((3, 2))

    for i in range(0, images):
        
        path = (str(i + 1) + '.exr')
        depth = exr_to_depth(path, background=False)
        camera_pose = np.reshape(np.array(read_csv(file='frames.csv', field='Matrix', row_number=i), dtype=float), (4, 4))

        # Compute camera view frustum and extend convex hull
        view_frust_pts = get_view_frustum(depth, camera_intrinsics, camera_pose)
        bounds[:, 0] = np.minimum(bounds[:, 0], np.amin(view_frust_pts, axis=1))
        bounds[:, 1] = np.maximum(bounds[:, 1], np.amax(view_frust_pts, axis=1))

    print("Initializing voxel volume...")
    tsdf_vol = TSDFVolume(bounds, voxel_size=0.013)

    # Loop through RGB-D images and fuse them together
    t0_elapse = time.time()

    for i in range(0, images):
        print("Fusing frame %d/%d" % (i + 1, images))

        # Read RGB-D image and camera pose
        path = (str(i + 1) + '.exr')
        color_image = exr_to_image(path)
        depth = exr_to_depth(path, background=False)

        camera_pose = np.reshape(np.array(read_csv(file='frames.csv', field='Matrix', row_number=i), dtype=float), (4, 4))

        # Integrate observation into voxel volume (assume color aligned with depth)
        tsdf_vol.integrate(color_image, depth, camera_intrinsics, camera_pose, obs_weight=1.)

    fps = images / (time.time() - t0_elapse)
    print("Average FPS: {:.2f}".format(fps))

    # Get mesh from voxel volume and save to disk (can be viewed with Meshlab)
    print("Saving mesh to mesh.ply...")
    verts, faces, norms, colors = tsdf_vol.get_mesh()

    meshwrite('mesh.ply', verts, faces, norms, colors)

    # Get point cloud from voxel volume and save to disk (can be viewed with Meshlab)
    # print("Saving point cloud to pc.ply...")
    # point_cloud = tsdf_vol.get_point_cloud()
    # fusion.pcwrite("pc.ply", point_cloud)


In [0]:
fuse()

Estimating voxel volume bounds...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


Initializing voxel volume...
Voxel volume size: 1140 x 348 x 1163 - # points: 461,385,360
Fusing frame 1/4
Fusing frame 2/4
Fusing frame 3/4
Fusing frame 4/4
Average FPS: 1.11
Saving mesh to mesh.ply...


In [0]:
show_mesh('mesh')

geometry::PointCloud with 441767 points.
